## ch08
- https://github.com/thampiman/interpretable-ai-book/blob/master/Chapter_08/chapter_08_fairness.ipynb

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.10.05</div>
<div style="text-align: right"> Last update: 2023.10.05</div>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [2]:
from tqdm.notebook import tqdm

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [3]:
from sklearn.model_selection import train_test_split
import shap

import pickle 

from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


이전 섹션에서는 랜덤 포레스트 모델을 학습시켜 급여를 예측했는데,  
이 모델의 목적은 각 성인이 5만 달러 이상의 소득을 올릴지 여부라는 이분법적 결과를 결정하는 것이었습니다.  

하지만 이러한 예측이 성별, 인종 등 다양한 보호 대상 그룹에 대해 공정하게 이루어졌을까요?   
공정성에 대한 다양한 개념의 정의를 공식화하기 위해 이 모델에 의한 예측과 공정성에 필요한 관련 측정값에 대한 간단한 그림을 살펴봅시다.   
그림 8.10은 2차원 평면에 투영된 모델에 의한 예측을 보여주는 그림입니다.  
랜덤 포레스트 모델은 2차원 평면을 두 개의 반으로 분할하여 긍정적인 예측(오른쪽 반)과 부정적인 예측(왼쪽 반)을 구분합니다.  
성인 20명의 행동 라벨도 이 2-D 평면에 투영되었습니다.  
2-D 평면에서 실제 라벨의 위치는 중요하지 않습니다.  
중요한 것은 라벨이 왼쪽 절반(모델이 음수, 즉 0을 예측하는 경우)에 있는지 오른쪽 절반(모델이 양수, 즉 1을 예측하는 경우)에 있는지의 여부입니다.  
실제 양수 레이블은 원으로 표시되고 실제 음수 레이블은 삼각형으로 표시됩니다.

![Alt text](image-7.png)

- 실제 양수 레이블 - 데이터 집합의 기준값 레이블이 양수인 데이터 포인트입니다. 그림 8.10에서는 데이터 집합에서 연간 소득이 5만 달러 이상인 성인이 원으로 표시되어 있습니다. 원을 세어보면 실제 양성 레이블의 수는 12개입니다.

- 실제 음수 레이블 - 데이터 집합의 기준값 레이블이 음수인 데이터 포인트입니다. 그림 8.10에서는 데이터 집합에서 연간 소득이 5만 달러 이하인 성인이 삼각형으로 표시되어 있습니다. 따라서 실제 음수 레이블의 수는 8개입니다.

- 예측된 양수 - 모델이 긍정적인 결과를 예측하는 데이터 포인트 그림 8.10에서 2차원 평면의 오른쪽 절반에 속하는 데이터 포인트는 양수 예측을 합니다. 해당 영역에 속하는 데이터 포인트는 10개입니다. 따라서 예측된 양성 측정값은 10입니다.

- 예측된 음수 - 모델이 음수 결과를 예측하는 데이터 포인트로, 그림 8.10에서는 2-D 평면의 왼쪽 절반에 속하는 포인트이며 예측된 음수 측정값도 10입니다.

- 진양성 - 그림 8.10에서 진양성은 2-D 평면의 오른쪽 절반에 해당하는 원입니다. 이는 본질적으로 모델이 양수로 예측하는 데이터 포인트이며 실제 레이블도 양수입니다. 이러한 원은 8개가 있으므로 진양성의 수는 8개입니다. 혼동 행렬에서도 이를 구할 수 있는데, 여기서 진양성은 모델이 1을 예측하고 실제 레이블이 1인 경우입니다.

- true negative : 반면에 진정한 네거티브는 2D 평면의 왼쪽 절반에 해당하는 삼각형입니다. 모델이 음수로 예측하고 실제 레이블도 음수인 데이터 포인트입니다. 그림 8.10에서 실제 네거티브의 수는 6입니다. 혼동 행렬에서 모델이 0을 예측하고 실제 레이블이 0인 경우를 볼 수 있습니다.   

- false positive - 오탐은 그림 8.10에서 평면의 오른쪽 절반에 해당하는 삼각형입니다. 모델이 양수로 예측하지만 실제 레이블은 음수인 데이터 포인트입니다. 그림에서 오탐의 수는 2입니다. 혼동 행렬에서 모델은 1을 예측하지만 실제 레이블은 0인 경우입니다.

- false negative - 오탐은 2차원 평면의 왼쪽 절반에 해당하는 원입니다. 기본적으로 모델이 음수로 예측하지만 실제 레이블은 양수인 데이터 포인트입니다. 그림 8.10의 왼쪽 절반에 4개의 원이 있으므로 오탐의 수는 4입니다. 혼동 행렬에서 보면 모델은 0을 예측하지만 실제 레이블은 1인 경우입니다.

Demographic parity

우리가 고려할 공정성의 첫 번째 개념은 인구통계학적 평등입니다.  
인구통계학적 동등성 개념은 독립성, 통계적 동등성, 법적으로는 이질적 영향이라고도 불립니다.  
이 개념은 모델에서 서로 다른 보호 대상 그룹에 대한 양수 예측값에 평등성이 존재한다고 주장합니다.  
그림8.11의 예시를 살펴보겠습니다.  
그림에서 20명의 성인은 그림 8.10에서 보았듯이 보호 대상 성별 그룹에 따라 각각 하나씩 A와 B의 두 그룹으로 분리되어 있습니다.  
그룹 A는 남성 성인으로 구성되며 2D 평면에 10개의 데이터 포인트가 있습니다.  
그룹 B는 2-D 평면에 10개의 데이터 포인트가 있는 여성 성인으로 구성되며, 그림 8.11의 그림을 기반으로 앞서 설명한 기본 측정값을 계산할 수 있습니다.  
남성 성인의 경우 실제 양성 6개, 실제 음성 4개, 예측 양성 5개, 예측 음성 5개가 있습니다.  
여성 성인의 경우 실제 양성/음성 및 예측 양성/음성이 남성 성인과 동일하다는 것을 알 수 있습니다.  
남성 성인과 여성 성인 모두의 긍정률은 모델이 긍정으로 예측한 각 그룹 내 성인의 비율입니다.  
그림 8.11에서 남성과 여성 사용자 모두의 양성 비율이 50%로 동일하다는 것을 알 수 있습니다. 따라서 두 그룹 간에 인구통계학적 동등성이 있다고 주장할 수 있습니다.

![Alt text](image-8.png)

이를 실용적인 관점에서 살펴봅시다.  
주택 대출과 같은 희소 자원을 할당하는 데 모델 예측이 사용된다고 가정해 보겠습니다.  
또한 연소득이 5만 달러 이상인 성인이 주택을 구입하고 대출금을 갚을 가능성이 더 높다고 가정해 봅시다.  
주택 대출 신청과 같은 결정이 모델 예측에 따라 이루어지고 연소득이 5만 달러 이상인 성인에게 대출이 승인되는 경우,  
인구통계학적 형평성은 남성과 여성 성인 모두에게 동일한 비율로 대출이 승인되도록 보장합니다.  
인구통계학적 평등은 모델이 남성과 여성 성인의 급여가 동일한 확률로 $50,000 이상이라고 예측한다고 가정합니다.

이제 인구통계학적 평등성을 좀 더 공식적으로 정의하고 이 정의를 사용하여 다음과 같이 확인해 보겠습니다.  
이 개념을 사용하여 랜덤 포레스트 모델이 공정한지 확인해 보겠습니다.  
모델 예측을 ŷ로, 보호 그룹 변수를 z로 표현해 보겠습니다.  
그룹은 여성 성인의 경우 0, 남성 성인의 경우 1의 두 가지 가능한 값을 가질 수 있습니다.
남성 성인. 인종 보호 그룹의 경우에도 변수 z에 가능한 두 가지 값이 있을 수 있습니다.   
흑인 성인의 경우 0, 백인 성인의 경우 1입니다. 인구통계학적 동등성을 유지하려면
모델이 한 보호 그룹에 대해 양성을 예측할 확률이 비슷하거나 같아야 합니다.   
다른 보호 대상 그룹에 대해 모델이 양성을 예측할 확률과 비슷하거나 같아야 합니다.  

확률 측정값은 비율이 임계값 τ1과 τ2 사이에 있는 경우 유사합니다.  
임계값은 일반적으로 각각 0.8과 1.2입니다. 임계값은 0.8과 1.2입니다.  
임계값은 0.8과 1.2로, 다음 공식에 표시된 바와 같이 법률 문헌에 나와 있는 80% 규칙에 가깝습니다.  
다음 방정식. 비율이 1이면 확률 측정값은 동일합니다:  

![Alt text](image-9.png)

이제 범주형이지만 두 개 이상의 값이 있는 보호된 그룹 기능에 이 정의를 어떻게 사용할 수 있을까요?  
이 예에서는 백인과 흑인의 두 가지 인종만 고려했습니다.  
데이터 집합에 더 많은 인종이 있다면 어떨까요?  
개인이 여러 인종으로 식별되는 이중 인종일 수 있습니다.  
다중 인종으로 식별되는 개인에 대한 차별이 없도록 이를 별도의 인종으로 취급합니다.  
두 개 이상의 인종이 있는 시나리오에서는 각 인종에 대한 인구통계학적 불균형 비율 메트릭을 정의하고 z = 0은 관심 인종을 나타내고 z = 1은 다른 모든 인종을 나타내는 일대일 전략을 취합니다.  
다인종인 개인은 여러 그룹에 속할 수 있다는 점에 유의하세요.  
그런 다음 다른 모든 인종과 비교할 때 모든 인종에 대해 인구통계학적 동등성 비율이 비슷한지 확인해야 합니다.  
연령과 같이 연속적인 보호 그룹 특성의 경우 어떻게 해야 할까요?  
이 경우 연속형 특징을 불연속형 그룹으로 분할한 다음 하나 대 전체 전략을 적용해야 합니다.

이제 정의가 적용되었으므로 랜덤 포레스트 모델이 공정한지 확인해 보겠습니다.  다음 코드 스니펫은 인구 통계학적 패리티 개념을 사용하여 모델을 평가합니다:

### Load prepared data and models

In [4]:
# with open('df_X.pkl', 'rb') as f:
#     df_X = pickle.load(f)
# with open('df_y.pkl', 'rb') as f:
#     df_y = pickle.load(f)
# with open('df_X_display.pkl', 'rb') as f:
#     df_X_display = pickle.load(f)
# with open('X_train.pkl', 'rb') as f:
#     X_train = pickle.load(f)
# with open('X_test.pkl', 'rb') as f:
#     X_test = pickle.load(f)
# with open('y_train.pkl', 'rb') as f:
#     y_train = pickle.load(f)
# with open('y_test.pkl', 'rb') as f:
#     y_test = pickle.load(f)
# with open('shap_values.pkl', 'rb') as f:
#     shap_values = pickle.load(f)
# with open('shap_explainer.pkl', 'rb') as f:
#     explainer = pickle.load(f)
# with open('adult_model.pkl', 'rb') as f:
#     adult_model = pickle.load(f)